In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Mission 1:

* Calculate the Average Rating according to current comments and compare it with the existing average rating.

In [2]:
path = "/Users/gokhanersoz/Desktop/VBO_Dataset/amazon_review.csv"

amazon_review = pd.read_csv(path)

In [75]:
df = amazon_review.copy()
print("DataFrame Shape : {}".format(df.shape))

DataFrame Shape : (4915, 12)


In [76]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0


In [77]:
# Are there any missing values?
# Can be ignored !!!

na_values = df.isnull().sum()
na_values = pd.DataFrame(na_values)
na_values.columns = ["Na_Values"]
na_values = na_values[na_values["Na_Values"] > 0 ]
na_values

,Na_Values
reviewerName,1
reviewText,1


In [78]:
dtypes = pd.DataFrame(df.dtypes).reset_index()
dtypes.columns = ["Name", "Dtype"]
dtypes

,Name,Dtype
0,reviewerID,object
1,asin,object
2,reviewerName,object
3,helpful,object
4,reviewText,object
5,overall,float64
6,summary,object
7,unixReviewTime,int64
8,reviewTime,object
9,day_diff,int64


In [79]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0


In [80]:
# We changed our Review variable to datetime....

df["reviewTime"] = pd.to_datetime(df["reviewTime"])
df.dtypes

reviewerID                object
asin                      object
reviewerName              object
helpful                   object
reviewText                object
overall                  float64
summary                   object
unixReviewTime             int64
reviewTime        datetime64[ns]
day_diff                   int64
helpful_yes                int64
total_vote                 int64
dtype: object

In [81]:
# unixReviewTime variables review Time has the same meaning... 
# With pd.to_datetime(df["unix Review Time"] , unit = "s") we can see that ....

time_df = pd.DataFrame()
time_df["reviewTime"] = pd.to_datetime(df["reviewTime"])
time_df["unixReviewTime"] = pd.to_datetime(df["unixReviewTime"], unit="s")
time_df.dtypes

reviewTime        datetime64[ns]
unixReviewTime    datetime64[ns]
dtype: object

In [82]:
time_df.head()

,reviewTime,unixReviewTime
0,2014-07-23,2014-07-23
1,2013-10-25,2013-10-25
2,2012-12-23,2012-12-23
3,2013-11-21,2013-11-21
4,2013-07-13,2013-07-13


In [83]:
nunique = pd.DataFrame(df.nunique()).reset_index()
nunique.columns = ["Names" , "Nunique"]
nunique.sort_values("Nunique", ascending = False)

,Names,Nunique
0,reviewerID,4915
4,reviewText,4912
2,reviewerName,4594
6,summary,3885
7,unixReviewTime,690
8,reviewTime,690
9,day_diff,690
3,helpful,42
11,total_vote,26
10,helpful_yes,23


In [84]:
# So I'll cover the required columns...
# Daydiff calculated for us but we recalculate ... (1 more than max day ...)
# [1,2] : Here the first one represents helpful_yes while the other one represents total_vote..
# So df["total_vote"] - df['helpful_yes'] = df["helpful_no"]

df[df["total_vote"] > 0][["helpful","helpful_yes","total_vote"]].head()

,helpful,helpful_yes,total_vote
8,"[1, 1]",1,1
17,"[0, 1]",0,1
26,"[1, 2]",1,2
28,"[1, 1]",1,1
37,"[1, 1]",1,1


In [85]:
df["helpful_no"] = df["total_vote"] - df["helpful_yes"]
df = df[["reviewText", "reviewTime", "overall", "summary", "helpful_yes","helpful_no","total_vote"]]
df.head()

,reviewText,reviewTime,overall,summary,helpful_yes,helpful_no,total_vote
0,No issues.,2014-07-23,4.0,Four Stars,0,0,0
1,"Purchased this for my device, it worked as adv...",2013-10-25,5.0,MOAR SPACE!!!,0,0,0
2,it works as expected. I should have sprung for...,2012-12-23,4.0,nothing to really say....,0,0,0
3,This think has worked out great.Had a diff. br...,2013-11-21,5.0,Great buy at this price!!! *** UPDATE,0,0,0
4,"Bought it with Retail Packaging, arrived legit...",2013-07-13,5.0,best deal around,0,0,0


In [86]:
print("Max Overall : {}".format(df["overall"].max()) )
print("Min Overall : {}".format(df["overall"].min()) )

Max Overall : 5.0
Min Overall : 1.0


In [87]:
# Is the average really for this product?

print("Average Overall : {}".format(df["overall"].mean()))

Average Overall : 4.587589013224822


In [88]:
print("Max Time : {}".format(df["reviewTime"].max()))
print("Min Time : {}".format(df["reviewTime"].min()))

Max Time : 2014-12-07 00:00:00
Min Time : 2012-01-09 00:00:00


In [89]:
import datetime as dt
current_date = pd.to_datetime("2014-12-8")
current_date

Timestamp('2014-12-08 00:00:00')

In [97]:
df["days"] = (current_date - df["reviewTime"]).dt.days
df.head()

,reviewText,reviewTime,overall,summary,helpful_yes,helpful_no,total_vote,days
0,No issues.,2014-07-23,4.0,Four Stars,0,0,0,138
1,"Purchased this for my device, it worked as adv...",2013-10-25,5.0,MOAR SPACE!!!,0,0,0,409
2,it works as expected. I should have sprung for...,2012-12-23,4.0,nothing to really say....,0,0,0,715
3,This think has worked out great.Had a diff. br...,2013-11-21,5.0,Great buy at this price!!! *** UPDATE,0,0,0,382
4,"Bought it with Retail Packaging, arrived legit...",2013-07-13,5.0,best deal around,0,0,0,513


In [99]:
df.sort_values("reviewTime", ascending = False).head()

,reviewText,reviewTime,overall,summary,helpful_yes,helpful_no,total_vote,days
2793,Great Price,2014-12-07,5.0,16 GB,0,0,0,1
4364,Best card for an 808 #26. Gives you 2.5 charge...,2014-12-07,5.0,Best card for an 808 #26,0,0,0,1
2629,Good transfer speed,2014-12-07,5.0,Five Stars,0,0,0,1
1494,Work perfectly in my camera. Reads and writes ...,2014-12-07,5.0,Great buy,0,0,0,1
3741,Great,2014-12-07,5.0,Five Stars,0,0,0,1


In [102]:
# Let's format it by time...
# We have weighted by time intervals ...

def time_based_weighted_average(dataframe , w1 = 28 , w2 = 26, w3 = 24, w4 = 22):
    
    results = \
    df.loc[ df["days"] <= 60, "overall" ].mean() * w1 / 100 + \
    df.loc[ (df["days"] > 60) & (df["days"] <=120), "overall"].mean() * w2 / 100 + \
    df.loc[ (df["days"] > 120) & (df["days"] <=180) , "overall"].mean() * w3 / 100 + \
    df.loc[ (df["days"] > 180),"overall"].mean() * w4 / 100
    
    return results

In [103]:
print("General Overall : {}".format(df.overall.mean())) 
print("By Time : {}".format(time_based_weighted_average(df)))

General Overall : 4.587589013224822
By Time : 4.68728061302512


## Mission 2:
* Specify 20 reviews to be displayed on the product detail page for the product.

In [105]:
################################################ #
# Wilson Lower Bound Score
################################################ #


# Helpful / Not Found
# Liked / Disliked
# Thumbs Up/ Thumbs Down
# up /down

# p : up rate

# For example, if there is 600 up to 400 down interaction, what is the up rate?
# 600/1000 is our 0.6 up rate. so p is our hat value.


# It will be at this rate with 0.05 deviation between 0.5 and 0.7 instead of 0.6 !!!

def wilson_lower_bound(up, down, confidence=0.95):
    
    """
    Calculate Wilson Lower Bound Score

     - The lower limit of the confidence interval to be calculated for the Bernoulli parameter p is accepted as the WLB score.
     - The score to be calculated is used for product ranking.
     - Note:
     If the scores are between 1-5, 1-3 are marked as negative, 4-5 as positive and can be adjusted to Bernoulli.
     This brings with it some problems. For this reason, it is necessary to make a bayesian average rating.
    
    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    import scipy.stats as st
    import math
    
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [106]:
def score_up_down_diff(up,down):
    return up - down

In [107]:
def score_average_rating(up,down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [108]:
#If Helpful hadn't been given to us in pieces we could have separated it by doing so....

# df.helpful = df.helpful.apply(lambda x : x.strip("[]").split(","))
# df["helpful_yes"] = df.helpful.apply(lambda x : x[0]).astype("int64")
# df["total_reviews"] = df.helpful.apply(lambda x : x[1]).astype("int64")
# df["helpful_no"] = df["total_reviews"] - df["helpful_yes"]

In [109]:
# Score_pos_neg_diff
df["score_pos_neg_diff"] = df.apply(lambda x : score_up_down_diff(x["helpful_yes"],x["helpful_no"]), axis = 1)

# Score_average_rating
df["score_average_rating"] = df.apply(lambda x : score_average_rating(x["helpful_yes"],x["helpful_no"]), axis = 1)

# wilson_lower_bound
df["wilson_lower_bound"] = df.apply(lambda x : wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis = 1)

In [110]:
# Ranking by score_pos_neg_diff and score_average_rating can give us wrong information.
# Or it may cause the ones that should stand out not to come out...
# Thus, we can be hesitant to buy or get frustrated because of the information we can't get about the product...

df.sort_values("wilson_lower_bound" , ascending = False).head(20)

,reviewText,reviewTime,overall,summary,helpful_yes,helpful_no,total_vote,days,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,2013-01-05,5.0,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,702,1884,0.966337,0.957544
3449,I have tested dozens of SDHC and micro-SDHC ca...,2012-09-26,5.0,Top of the class among all (budget-priced) mic...,1428,77,1505,803,1351,0.948837,0.936519
4212,NOTE: please read the last update (scroll to ...,2013-05-08,1.0,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,579,1442,0.925620,0.912139
317,"If your card gets hot enough to be painful, it...",2012-02-09,1.0,"Warning, read this!",422,73,495,1033,349,0.852525,0.818577
4672,Sandisk announcement of the first 128GB micro ...,2014-07-03,5.0,Super high capacity!!! Excellent price (on Am...,45,4,49,158,41,0.918367,0.808109
1835,Bought from BestBuy online the day it was anno...,2014-02-28,5.0,I own it,60,8,68,283,52,0.882353,0.784651
3981,The last few days I have been diligently shopp...,2012-10-22,5.0,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,777,85,0.805755,0.732136
3807,I bought this card to replace a lost 16 gig in...,2013-02-27,3.0,"Good buy for the money but wait, I had an issue!",22,3,25,649,19,0.880000,0.700442
4306,"While I got this card as a ""deal of the day"" o...",2012-09-06,5.0,Awesome Card!,51,14,65,823,37,0.784615,0.670334
4596,Hi:I ordered two card and they arrived the nex...,2012-09-22,1.0,Designed incompatibility/Don't support SanDisk,82,27,109,807,55,0.752294,0.663595
